#**I - Simple EQL**
  including the following symbolic operations {multiplication, sin, sign, ln}

## **I.1 - Base Functions**

In [ ]:
""" Functions for use with Symbolic Regression """

import numpy as np
import torch
import sympy as sym

class BaseFunction :
  """Abstract parent class for primitive symbolic functions"""

  def __init__(self,norm=1) :
    self.norm=norm


  def sp(self,x) :
    return None

  def torch(self,x) :
    return None

  def name(self,x) :
    """Sympy to String"""
    return str(self.sp)

  def np(self,x) :
    """Sympy to Numpy"""
    z=sp.symbols('z')
    return sym.utilities.lambdify(z,self.sp(z),'numpy')(x)


class Sin(BaseFunction) :
  def torch(self, x) :
    return torch.sin(4*np.pi*x) /self.norm

  def sp(self, x) :
    return sym.sin(4*np.pi*x) /self.norm

class Log(BaseFunction) : #do not forget that we work in R+
  def torch(self, x) :
    return torch.log(torch.abs(x)) /self.norm

  def sp(self, x) :
    return sym.log(sym.Abs(x)) /self.norm

class Sign(BaseFunction) :
  def torch(self, x) :
    return torch.sign(x) /self.norm

  def sp(self, x) :
    return sym.sign(x) /self.norm


class Identity(BaseFunction) :
  def torch(self, x) :
    return x /self.norm

  def sp(self, x) :
    return x /self.norm

class Constant(BaseFunction) :
  def torch(self, x) :
    return torch.ones_like(x)

  def sp(self, x) :
    return 1

  def np(self,x) :
    return np.ones_like


class Square(BaseFunction) :
  def torch(self, x) :
    return torch.square(x) /self.norm

  def sp(self, x) :
    return x**2 /self.norm


class BaseFunction2 :
  """Abstract parent class for primitive symbolic functions WITH 2 VARIABLES"""

  def __init__(self,norm=1) :
    self.norm=norm

  def sp(self,x,y) :
    return None

  def torch(self,x,y) :
    return None

  def name(self,x,y) :
    """Sympy to String"""
    return str(self.sp)

  def np(self,x,y) :
    """Sympy to Numpy"""
    a,b=sp.symbols('z w')
    return sym.utilities.lambdify([a,b],self.sp(a,b),'numpy')(x,y)

class Product(BaseFunction2) :
  def __init__(self,norm=0.1) :
    super().__init__(norm=norm) # do not forget how to use super

  def torch(self,x,y) :
    return x*y / self.norm

  def sp(self,x,y) :
    return x*y / self.norm


def count_inputs(funcs) :
  i=0
  for func in funcs :
    if isinstance(func,BaseFunction) :
      i+=1
    elif isinstance(func,BaseFunction2) :
      i+=2
  return i

def count_double(funcs) :
  i=0
  for func in funcs :
    if isinstance(func,BaseFunction2) :
      i+=1
  return i


# default_func= [
#    Sin()
#    ...
#    ]

default_func = [      # The number of times each symbolic primitive appear fix its probability of being chosen by the network
    *[Constant()] * 2,
    *[Identity()] * 4,
    *[Square()] * 4,
    *[Sin()] * 2,
    *[Sign()] * 2,
    *[Log()] * 2,
    *[Product()] * 2, #2 inputs = 1 output at the end of default_func
]

## **I.2 - EQL Network**


###*Without Regularization*

In [ ]:
"""Contains the symbolic regression neural network architecture."""
import numpy as np
import torch
import torch.nn as nn
#from utils import functions as functions


class SymbolicLayer(nn.Module):
    """Neural network layer for symbolic regression where activation functions correspond to primitive functions.
    Can take multi-input activation functions (like multiplication)"""
    def __init__(self, funcs=None, initial_weight=None, init_stddev=0.1, in_dim=None):
        """
        funcs: List of activation functions, using utils.functions
        initial_weight: (Optional) Initial value for weight matrix
        variable: Boolean of whether initial_weight is a variable or not
        init_stddev: (Optional) if initial_weight isn't passed in, this is standard deviation of initial weight
        """
        super().__init__()

        if funcs is None:
            funcs = default_func
        self.initial_weight = initial_weight
        self.W = None       # Weight matrix
        self.built = False  # Boolean whether weights have been initialized

        self.output = None  # tensor for layer output
        self.n_funcs = len(funcs)                       # Number of activation functions (and number of layer outputs)
        self.funcs = [func.torch for func in funcs]     # Convert functions to list of PyTorch functions
        self.n_double = count_double(funcs)   # Number of activation functions that take 2 inputs
        self.n_single = self.n_funcs - self.n_double    # Number of activation functions that take 1 input

        self.out_dim = self.n_funcs + self.n_double

        if self.initial_weight is not None:     # use the given initial weight
            self.W = nn.Parameter(self.initial_weight.clone().detach())  # copies
            self.built = True
        else:
            self.W = torch.normal(mean=0.0, std=init_stddev, size=(in_dim, self.out_dim))

    def forward(self, x):  # used to be __call__
        """Multiply by weight matrix and apply activation units"""

        g = torch.matmul(x, self.W)         # shape = (?, self.size)
        self.output = []

        in_i = 0    # input index
        out_i = 0   # output index
        # Apply functions with only a single input
        while out_i < self.n_single:
            self.output.append(self.funcs[out_i](g[:, in_i]))
            in_i += 1
            out_i += 1
        # Apply functions that take 2 inputs and produce 1 output
        while out_i < self.n_funcs:
            self.output.append(self.funcs[out_i](g[:, in_i], g[:, in_i+1]))
            in_i += 2
            out_i += 1

        self.output = torch.stack(self.output, dim=1)

        return self.output

    def get_weight(self):
        return self.W.cpu().detach().numpy()

    def get_weight_tensor(self):
        return self.W.clone()


class SymbolicNet(nn.Module):
    """Symbolic regression network with multiple layers. Produces one output."""
    def __init__(self, symbolic_depth, funcs=None, initial_weights=None, init_stddev=0.1):
        super(SymbolicNet, self).__init__()

        self.depth = symbolic_depth     # Number of hidden layers
        self.funcs = funcs
        layer_in_dim = [1] + self.depth*[len(funcs)]

        if initial_weights is not None:
            layers = [SymbolicLayer(funcs=funcs, initial_weight=initial_weights[i], in_dim=layer_in_dim[i])
                      for i in range(self.depth)]
            self.output_weight = nn.Parameter(initial_weights[-1].clone().detach())

        else:
            # Each layer initializes its own weights
            if not isinstance(init_stddev, list):
                init_stddev = [init_stddev] * self.depth
            layers = [SymbolicLayer(funcs=self.funcs, init_stddev=init_stddev[i], in_dim=layer_in_dim[i])
                      for i in range(self.depth)]

            # Initialize weights for last layer (without activation functions)
            self.output_weight = nn.Parameter(torch.rand((layers[-1].n_funcs, 1)))
        self.hidden_layers = nn.Sequential(*layers)

    def forward(self, input):
        h = self.hidden_layers(input)   # Building hidden layers
        return torch.matmul(h, self.output_weight)  # Final output (no activation units) of network

    def get_weights(self):
        """Return list of weight matrices"""
        # First part is iterating over hidden weights. Then append the output weight.
        return [self.hidden_layers[i].get_weight() for i in range(self.depth)] + \
               [self.output_weight.cpu().detach().numpy()]

    def get_weights_tensor(self):
        """Return list of weight matrices as tensors"""
        return [self.hidden_layers[i].get_weight_tensor() for i in range(self.depth)] + \
               [self.output_weight.clone()]


###*With Regularization*

In [ ]:

class SymbolicLayerL0(SymbolicLayer):
    def __init__(self, in_dim=None, funcs=None, initial_weight=None, init_stddev=0.1,
                 bias=False, droprate_init=0.5, lamba=1.,
                 beta=2 / 3, gamma=-0.1, zeta=1.1, epsilon=1e-6):
        super().__init__(in_dim=in_dim, funcs=funcs, initial_weight=initial_weight, init_stddev=init_stddev)

        self.droprate_init = droprate_init if droprate_init != 0 else 0.5
        self.use_bias = bias
        self.lamba = lamba
        self.bias = None
        self.in_dim = in_dim
        self.eps = None

        self.beta = beta
        self.gamma = gamma
        self.zeta = zeta
        self.epsilon = epsilon

        if self.use_bias:
            self.bias = nn.Parameter(0.1 * torch.ones((1, self.out_dim)))
        self.qz_log_alpha = nn.Parameter(torch.normal(mean=np.log(1 - self.droprate_init) - np.log(self.droprate_init),
                                                      std=1e-2, size=(in_dim, self.out_dim)))

    def quantile_concrete(self, u):
        """Quantile, aka inverse CDF, of the 'stretched' concrete distribution"""
        y = torch.sigmoid((torch.log(u) - torch.log(1.0 - u) + self.qz_log_alpha) / self.beta)
        return y * (self.zeta - self.gamma) + self.gamma

    def sample_u(self, shape, reuse_u=False):
        """Uniform random numbers for concrete distribution"""
        if self.eps is None or not reuse_u:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.eps = torch.rand(size=shape).to(device) * (1 - 2 * self.epsilon) + self.epsilon
        return self.eps

    def sample_z(self, batch_size, sample=True):
        """Use the hard concrete distribution as described in https://arxiv.org/abs/1712.01312"""
        if sample:
            eps = self.sample_u((batch_size, self.in_dim, self.out_dim))
            z = self.quantile_concrete(eps)
            return torch.clamp(z, min=0, max=1)
        else:  # Mean of the hard concrete distribution
            pi = torch.sigmoid(self.qz_log_alpha)
            return torch.clamp(pi * (self.zeta - self.gamma) + self.gamma, min=0.0, max=1.0)

    def get_z_mean(self):
        """Mean of the hard concrete distribution"""
        pi = torch.sigmoid(self.qz_log_alpha)
        return torch.clamp(pi * (self.zeta - self.gamma) + self.gamma, min=0.0, max=1.0)

    def sample_weights(self, reuse_u=False):
        z = self.quantile_concrete(self.sample_u((self.in_dim, self.out_dim), reuse_u=reuse_u))
        mask = torch.clamp(z, min=0.0, max=1.0)
        return mask * self.W

    def get_weight(self):
        """Deterministic value of weight based on mean of z"""
        return self.W * self.get_z_mean()

    def loss(self):
        """Regularization loss term"""
        return torch.sum(torch.sigmoid(self.qz_log_alpha - self.beta * np.log(-self.gamma / self.zeta)))

    def forward(self, x, sample=True, reuse_u=False):
        """Multiply by weight matrix and apply activation units"""
        if sample:
            h = torch.matmul(x, self.sample_weights(reuse_u=reuse_u))
        else:
            w = self.get_weight()
            h = torch.matmul(x, w)

        if self.use_bias:
            h = h + self.bias

        # shape of h = (?, self.n_funcs)

        output = []
        # apply a different activation unit to each column of h
        in_i = 0  # input index
        out_i = 0  # output index
        # Apply functions with only a single input
        while out_i < self.n_single:
            output.append(self.funcs[out_i](h[:, in_i]))
            in_i += 1
            out_i += 1
        # Apply functions that take 2 inputs and produce 1 output
        while out_i < self.n_funcs:
            output.append(self.funcs[out_i](h[:, in_i], h[:, in_i + 1]))
            in_i += 2
            out_i += 1
        output = torch.stack(output, dim=1)
        return output


class SymbolicNetL0(nn.Module):
    """Symbolic regression network with multiple layers. Produces one output."""

    def __init__(self, symbolic_depth, in_dim=1, funcs=None, initial_weights=None, init_stddev=0.1):
        super(SymbolicNetL0, self).__init__()
        self.depth = symbolic_depth  # Number of hidden layers
        self.funcs = funcs

        layer_in_dim = [in_dim] + self.depth * [len(funcs)]
        if initial_weights is not None:
            layers = [SymbolicLayerL0(funcs=funcs, initial_weight=initial_weights[i],
                                      in_dim=layer_in_dim[i])
                      for i in range(self.depth)]
            self.output_weight = nn.Parameter(initial_weights[-1].clone().detach())
        else:
            # Each layer initializes its own weights
            if not isinstance(init_stddev, list):
                init_stddev = [init_stddev] * self.depth
            layers = [SymbolicLayerL0(funcs=funcs, init_stddev=init_stddev[i], in_dim=layer_in_dim[i])
                      for i in range(self.depth)]
            # Initialize weights for last layer (without activation functions)
            self.output_weight = nn.Parameter(torch.rand(size=(self.hidden_layers[-1].n_funcs, 1)) * 2)
        self.hidden_layers = nn.Sequential(*layers)

    def forward(self, input, sample=True, reuse_u=False):
        # connect output from previous layer to input of next layer
        h = input
        for i in range(self.depth):
            h = self.hidden_layers[i](h, sample=sample, reuse_u=reuse_u)

        h = torch.matmul(h, self.output_weight)     # Final output (no activation units) of network
        return h

    def get_loss(self):
        return torch.sum(torch.stack([self.hidden_layers[i].loss() for i in range(self.depth)]))

    def get_weights(self):
        """Return list of weight matrices"""
        # First part is iterating over hidden weights. Then append the output weight.
        return [self.hidden_layers[i].get_weight().cpu().detach().numpy() for i in range(self.depth)] + \
               [self.output_weight.cpu().detach().numpy()]

## **I.3 - Symbolic Expression of the EQL Network**

In [ ]:
"""
Generate a mathematical expression of the symbolic regression network (AKA EQL network) using SymPy. This expression
can be used to pretty-print the expression (including human-readable text, LaTeX, etc.). SymPy also allows algebraic
manipulation of the expression.
The main function is network(...)
There are several filtering functions to simplify expressions, although these are not always needed if the weight matrix
is already pruned.
"""

import sympy as sym

def apply_activation(W, funcs, n_double=0):
    """Given an (n, m) matrix W and (m) vector of funcs, apply funcs to W.

    Arguments:
        W:  (n, m) matrix
        funcs: list of activation functions (SymPy functions)
        n_double:   Number of activation functions that take in 2 inputs

    Returns:
        SymPy matrix with 1 column that represents the output of applying the activation functions.
    """
    W = sym.Matrix(W)
    if n_double == 0:
        for i in range(W.shape[0]):
            for j in range(W.shape[1]):
                W[i, j] = funcs[j](W[i, j])
    else:
        W_new = W.copy()
        out_size = len(funcs)
        for i in range(W.shape[0]):
            in_j = 0
            out_j = 0
            while out_j < out_size - n_double:
                W_new[i, out_j] = funcs[out_j](W[i, in_j])
                in_j += 1
                out_j += 1
            while out_j < out_size:
                W_new[i, out_j] = funcs[out_j](W[i, in_j], W[i, in_j+1])
                in_j += 2
                out_j += 1
        for i in range(n_double):
            W_new.col_del(-1)
        W = W_new
    return W


def sym_pp(W_list, funcs, var_names, threshold=0.01, n_double=0):
    """Pretty print the hidden layers (not the last layer) of the symbolic regression network

    Arguments:
        W_list: list of weight matrices for the hidden layers
        funcs:  list of lambda functions using sympy. has the same size as W_list[i][j, :]
        var_names: list of strings for names of variables
        threshold: threshold for filtering expression. set to 0 for no filtering.
        n_double:   Number of activation functions that take in 2 inputs

    Returns:
        Simplified sympy expression.
    """
    vars = []
    for var in var_names:
        if isinstance(var, str):
            vars.append(sym.Symbol(var))
        else:
            vars.append(var)
    expr = sym.Matrix(vars).T
    # W_list = np.asarray(W_list)
    for W in W_list:
        W = filter_mat(sym.Matrix(W), threshold=threshold)
        expr = expr * W
        expr = apply_activation(expr, funcs, n_double=n_double)
    # expr = expr * W_list[-1]
    return expr


def last_pp(eq, W):
    """Pretty print the last layer."""
    return eq * filter_mat(sym.Matrix(W))


def network(weights, funcs, var_names, threshold=0.01):
    """Pretty print the entire symbolic regression network.

    Arguments:
        weights: list of weight matrices for the entire network
        funcs:  list of lambda functions using sympy. has the same size as W_list[i][j, :]
        var_names: list of strings for names of variables
        threshold: threshold for filtering expression. set to 0 for no filtering.

    Returns:
        Simplified sympy expression."""
    n_double = count_double(funcs)
    funcs = [func.sp for func in funcs]

    expr = sym_pp(weights[:-1], funcs, var_names, threshold=threshold, n_double=n_double)
    expr = last_pp(expr, weights[-1])
    expr = expr[0, 0]
    return expr


def filter_mat(expr, threshold=0.01):
    """Sets all constants under threshold to 0
    TODO: Test"""
    for a in sym.preorder_traversal(expr):
        if isinstance(a, sym.Float) and a < threshold:
            expr = expr.subs(a, 0)
    return expr

## **I.4 - Test of the EQL Network**

In [ ]:
"""Trains the deep symbolic regression architecture on given functions to produce a simple equation that describes
the dataset. Uses L_0 regularization for the EQL network."""

import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from inspect import signature
import time


N_TRAIN = 100       # Size of training dataset
N_VAL = 100         # Size of validation dataset
DOMAIN = (-3, 3)    # Domain of dataset - range from which we sample x
# DOMAIN = np.array([[0, -1, -1], [1, 1, 1]])   # Use this format if each input variable has a different domain
N_TEST = 100        # Size of test dataset
DOMAIN_TEST = (-3, 3)   # Domain of test dataset - should be larger than training domain to test extrapolation
NOISE_SD = 0        # Standard deviation of noise for training dataset
var_names = ["x", "y", "z"]

# Standard deviation of random distribution for weight initializations.
init_sd_first = 0.1
init_sd_last = 1.0
init_sd_middle = 0.5
# init_sd_first = 0.5
# init_sd_last = 0.5
# init_sd_middle = 0.5
# init_sd_first = 0.1
# init_sd_last = 0.1
# init_sd_middle = 0.1


def generate_data(func, N, range_min=DOMAIN[0], range_max=DOMAIN[1]):
    """Generates datasets."""
    x_dim = len(signature(func).parameters)     # Number of inputs to the function, or, dimensionality of x
    x = (range_max - range_min) * torch.rand([N, x_dim]) + range_min
    y = torch.tensor([[func(*x_i)] for x_i in x])
    return x, y


class Benchmark:

    def __init__(self, n_layers=1, reg_weight=5e-3, learning_rate=1e-2,
                 n_epochs1=10001, n_epochs2=10001):
        """Set hyper-parameters"""
        self.activation_funcs = [
            *[Identity()] * 1,
            *[Sin()] * 1,
        ]

        self.n_layers = n_layers                # Number of hidden layers
        self.reg_weight = reg_weight            # Regularization weight
        self.learning_rate = learning_rate
        self.summary_step = 1000                # Number of iterations at which to print to screen
        self.n_epochs1 = n_epochs1
        self.n_epochs2 = n_epochs2

    def benchmark(self, func, func_name, trials):
        """Benchmark the EQL network on data generated by the given function. Print the results ordered by test error.

        Arguments:
            func: lambda function to generate dataset
            func_name: string that describes the function - this will be the directory name
            trials: number of trials to train from scratch. Will save the results for each trial.
        """

        print("Starting benchmark for function:\t%s" % func_name)
        print("==============================================")

        # Train network!
        expr_list, error_test_list = self.train(func, func_name, trials)

        # Sort the results by test error (increasing) and print them to file
        # This allows us to easily count how many times it fit correctly.
        error_expr_sorted = sorted(zip(error_test_list, expr_list))     # List of (error, expr)
        error_test_sorted = [x for x, _ in error_expr_sorted]   # Separating out the errors
        expr_list_sorted = [x for _, x in error_expr_sorted]    # Separating out the expr

    def train(self, func, func_name='', trials=1):
        """Train the network to find a given function"""

        use_cuda = torch.cuda.is_available()
        device = torch.device("cuda" if use_cuda else "cpu")
        print("Use cuda:", use_cuda, "Device:", device)

        x, y = generate_data(func, N_TRAIN)
        data, target = x.to(device), y.to(device)
        # x_val, y_val = generate_data(func, N_VAL)
        x_test, y_test = generate_data(func, N_TEST, range_min=DOMAIN_TEST[0], range_max=DOMAIN_TEST[1])
        test_data, test_target = x_test.to(device), y_test.to(device)

        # Setting up the symbolic regression network
        x_dim = len(signature(func).parameters)  # Number of input arguments to the function

        width = len(self.activation_funcs)
        n_double = count_double(self.activation_funcs)

        # Arrays to keep track of various quantities as a function of epoch
        loss_list = []          # Total loss (MSE + regularization)
        error_list = []         # MSE
        reg_list = []           # Regularization
        error_test_list = []    # Test error

        error_test_final = []
        eq_list = []

        for trial in range(trials):
            print("Training on function " + func_name + " Trial " + str(trial+1) + " out of " + str(trials))

            # reinitialize for each trial
            net = SymbolicNetL0(self.n_layers,
                                funcs=self.activation_funcs,
                                initial_weights=[
                                    # kind of a hack for truncated normal
                                    torch.fmod(torch.normal(0, init_sd_first, size=(x_dim, width + n_double)), 3),
                                    torch.fmod(torch.normal(0, init_sd_middle, size=(width, width + n_double)), 3),
                                    torch.fmod(torch.normal(0, init_sd_middle, size=(width, width + n_double)), 3),
                                    torch.fmod(torch.normal(0, init_sd_last, size=(width, 1)), 3)
                                ]).to(device)

            loss_val = np.nan
            while np.isnan(loss_val):
                # training restarts if gradients blow up
                criterion = nn.MSELoss()
                optimizer = optim.RMSprop(net.parameters(),
                                          lr=self.learning_rate * 10,
                                          alpha=0.9,  # smoothing constant
                                          eps=1e-10,
                                          momentum=0.0,
                                          centered=False)

                # adapative learning rate
                lmbda = lambda epoch: 0.1
                scheduler = optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lmbda)
                # for param_group in optimizer.param_groups:
                #     print("Learning rate: %f" % param_group['lr'])

                t0 = time.time()

                # 0th warmup stage, then 2 stages of training with decreasing learning rate
                for epoch in range(self.n_epochs1 + self.n_epochs2 + 2000):
                    optimizer.zero_grad()  # zero the parameter gradients
                    outputs = net(data)  # forward pass
                    mse_loss = criterion(outputs, target)

                    reg_loss = net.get_loss()
                    loss = mse_loss + self.reg_weight * reg_loss
                    loss.backward()
                    optimizer.step()

                    if epoch % self.summary_step == 0:
                        error_val = mse_loss.item()
                        reg_val = reg_loss.item()
                        loss_val = loss.item()
                        error_list.append(error_val)
                        reg_list.append(reg_val)
                        loss_list.append(loss_val)

                        with torch.no_grad():  # test error
                            test_outputs = net(test_data)
                            test_loss = F.mse_loss(test_outputs, test_target)
                            error_test_val = test_loss.item()
                            error_test_list.append(error_test_val)

                        print("Epoch: %d\tTotal training loss: %f\tTest error: %f" % (epoch, loss_val, error_test_val))

                        if np.isnan(loss_val):  # If loss goes to NaN, restart training
                            break

                    if epoch == 2000:
                        scheduler.step()  # lr /= 10
                    elif epoch == self.n_epochs1 + 2000:
                        scheduler.step()    # lr /= 10 again

                scheduler.step()  # lr /= 10 again

                t1 = time.time()

            tot_time = t1 - t0
            print(tot_time)

            # Print the expressions
            with torch.no_grad():
                weights = net.get_weights()
                expr = network(weights, self.activation_funcs, var_names[:x_dim])
                print(expr)

            error_test_final.append(error_test_list[-1])
            eq_list.append(expr)

        return eq_list, error_test_final


if __name__ == "__main__":

    kwargs = {
        "n_layers": 2,
        "reg_weight": 5e-6,
        "learning_rate": 1e-2,
        "n_epochs1": 2001,
        "n_epochs2": 2001
    }
    print(kwargs)

    bench = Benchmark(**kwargs)

    bench.benchmark(lambda x: np.sin(x), func_name="sin(x)", trials=5)




{'n_layers': 2, 'reg_weight': 5e-06, 'learning_rate': 0.01, 'n_epochs1': 2001, 'n_epochs2': 2001}
Starting benchmark for function:	sin(x)
Use cuda: False Device: cpu
Training on function sin(x) Trial 1 out of 5
Epoch: 0	Total training loss: 0.271112	Test error: 0.553456
Epoch: 1000	Total training loss: 0.103743	Test error: 0.023668
Epoch: 2000	Total training loss: 0.035727	Test error: 0.091404
Epoch: 3000	Total training loss: 0.002403	Test error: 0.002452
Epoch: 4000	Total training loss: 0.001970	Test error: 0.001726
Epoch: 5000	Total training loss: 0.000141	Test error: 0.000119
Epoch: 6000	Total training loss: 0.000045	Test error: 0.000337
17.108752489089966
0.0504331*x + 0.920519*sin(1.04194128263668*x)
Training on function sin(x) Trial 2 out of 5
Epoch: 0	Total training loss: 0.471166	Test error: 0.571676
Epoch: 1000	Total training loss: 0.029539	Test error: 0.037158
Epoch: 2000	Total training loss: 0.041985	Test error: 0.061174
Epoch: 3000	Total training loss: 0.001953	Test error: 